In [14]:
import csv
import requests
import datetime

"""
A simple program to print the result of a Prometheus query as CSV.
"""
def query_csv(q, comment):
    # if len(sys.argv) != 3:
    #     print('Usage: {0} http://prometheus:9090 a_query'.format(sys.argv[0]))
    #     sys.exit(1)

    response = requests.get('http://localhost:9090/api/v1/query',
            params={'query': q})
    results = response.json()['data']['result']

    # Build a list of all labelnames used.
    labelnames = set()
    for result in results:
        labelnames.update(result['metric'].keys())

    # Canonicalize
    labelnames.discard('__name__')
    labelnames = sorted(labelnames)

    with open('test.csv', mode='a') as f:
        writer = csv.writer(f)
        # Write the header,
        writer.writerow(['time:', str(datetime.datetime.now()), 'comment:',comment])
        writer.writerow(['name', 'timestamp', 'value'] + labelnames)

        # Write the samples.
        for result in results:
            l = [result['metric'].get('__name__', '')] + result['value']
            for label in labelnames:
                l.append(result['metric'].get(label, ''))
            writer.writerow(l)
            

In [15]:
query_csv('avg by (job, instance, mode) (rate(process_cpu_seconds_total[5m]))', 'test1 after setup')